# Содержание
1. [Обзор данных](#1)
2. [Подготовка данных](#2) 
3. [Обучение моделей](#3)
4. [Анализ результатов](#4)
5. [Общий вывод](#5)

# Предсказание стоимости жилья

**Цель исследования** - разработать систему, которая предсказывает медианную стоимость дома в жилом массиве.

**Задачи:**

- подготовить данные;
- построить две модели линейной регрессии на разных наборах данных:
  - используя все данные из файла,
  - используя только числовые переменные, исключив категориальные;
- сравнить результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2.

**Описание данных**

В колонках датасета с информацией о жилье в Калифорнии в 1990 году содержатся следующие данные:
- longitude — широта;
- latitude — долгота;
- housing_median_age — медианный возраст жителей жилого массива;
- total_rooms — общее количество комнат в домах жилого массива;
- total_bedrooms — общее количество спален в домах жилого массива;
- population — количество человек, которые проживают в жилом массиве;
- households — количество домовладений в жилом массиве;
- median_income — медианный доход жителей жилого массива;
- median_house_value — медианная стоимость дома в жилом массиве;
- ocean_proximity — близость к океану.



## Обзор данных <a id="1"></a>

### Подключение необходимых библиотек.

In [1]:
import pandas as pd 
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline, PipelineModel


from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator


### Инициализация локальной Spark-сессии.

In [2]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()


### Открытие и изучение данных.

In [3]:
server_path = '/datasets/'
local_path = ''
data = 'housing.csv'

try:
    data = spark.read.load(server_path + data, 
                                            format="csv", sep=",", inferSchema=True, header="true")

except: 
    data = spark.read.load(local_path + data, 
                                            format="csv", sep=",", inferSchema=True, header="true")  
    

In [4]:
data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [5]:
null_df = data.select([F.count(F.when(F.isnan(i) | \
                                          F.col(i).contains('NA') | \
                                          F.col(i).contains('NULL') | \
                                          F.col(i).isNull(), i)).alias(i) \
                                  for i in data.columns])

null_df.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



- Целевой признак -  колонка `median_house_value`. 
- Данные состоят из числовых и одного категориального столбца `ocean_proximity`. Типы данных `double` и `string` соответсвенно, изменений не требуется.
- В столбце `total_bedrooms` обнаружены пропуски.

## Подготовка данных <a id="2"></a>

1) Для удобства переименуем столбец с целевым признаком, присвоив значение `label`.

In [6]:
data = data.withColumnRenamed("median_house_value", "label")

In [7]:
data.select('label').show(5)

+--------+
|   label|
+--------+
|452600.0|
|358500.0|
|352100.0|
|341300.0|
|342200.0|
+--------+
only showing top 5 rows



2) Выделим числовые признаки.

In [8]:
columns_features_num = data.columns
columns_features_num.remove("ocean_proximity")
columns_features_num.remove("label")

3) Разделим данные на тестовую и обучающую выборку перед трасформацией.

In [9]:
training_set, test_set = data.randomSplit([0.8, 0.2], 1)
print(training_set.count(), test_set.count()) 

16507 4133


4) Создадим трансформер данных с помощью pipeline.

In [10]:
# Imputer (заполнение пропусков)
imp = Imputer(inputCols = ["total_bedrooms"],
              outputCols = ["total_bedrooms"]).setStrategy("median")

# VectorAssembler (векторизация числовых признаков)
vec = VectorAssembler(inputCols = columns_features_num,
                      outputCol = "vfeatures")

# StandardScaler (масштабирование числовых признаков)
sca = StandardScaler(inputCol = "vfeatures",
                     outputCol = "scaled",
                     withStd = True)

# numPipeline
numPipeline = Pipeline(stages = [imp, vec, sca])

# Indexer (индексация категориальных признаков)
ind = StringIndexer(inputCol = "ocean_proximity", 
                    outputCol = "ocean_proximity_indexed").setHandleInvalid("error")

# Encoder (ohe для категориальных признаков)
enc = OneHotEncoder(inputCols = ["ocean_proximity_indexed"],
                             outputCols = ["ocean_proximity_hot"])

# catPipeline
catPipeline = Pipeline(stages = [ind, enc])

# Pipeline Assembly
pipeline = Pipeline(stages = [numPipeline, catPipeline])

# Fit and transform
pipemodel = pipeline.fit(training_set)

data_train = pipemodel.transform(training_set)
data_test = pipemodel.transform(test_set)

data_train.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+--------------------+--------------------+-----------------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|           vfeatures|              scaled|ocean_proximity_indexed|ocean_proximity_hot|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+--------------------+--------------------+-----------------------+-------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147| 94600.0|     NEAR OCEAN|[-124.35,40.54,52...|[-62.197108857510...|                    2.0|      (4,[2],[1.0])|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797| 85800.0|     NEAR OCEAN|[-124.3,41.8,19.0

In [11]:
# Vector Assembly (объединение трансформированных категориальных и числовых данных)
newVec = VectorAssembler(inputCols = ["scaled", "ocean_proximity_hot"],
                         outputCol = "features")


In [12]:
datatrain = newVec.transform(data_train)
datatest = newVec.transform(data_test)

## Обучение моделей <a id="3"></a>

1) Обучение на всех данных файла.

In [13]:
lr = LinearRegression(labelCol='label', featuresCol='features')

model = lr.fit(datatrain) 

23/04/23 08:03:59 WARN Instrumentation: [3c5f3aa0] regParam is zero, which might cause numerical instability and overfitting.
23/04/23 08:03:59 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/04/23 08:03:59 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/04/23 08:04:00 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/04/23 08:04:00 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [14]:
print(model.coefficients)
print('rmse:', model.summary.rootMeanSquaredError)
print('mae:', model.summary.meanAbsoluteError)
print('r2', model.summary.r2)

[-52016.286939065474,-53204.63398449369,13203.921852460346,-12351.974169540867,29843.725328642802,-43053.56825741572,30367.197674049417,73930.74037788033,-187003.5633910455,-226431.96558833637,-181550.96693509695,-190113.1009098607]
rmse: 68711.58132075836
mae: 49752.55306255161
r2 0.6437946791912978


2) Обучение только на числовых данных.

In [15]:
lr2 = LinearRegression(labelCol='label', featuresCol='vfeatures')

model2 = lr2.fit(datatrain) 

23/04/23 08:04:02 WARN Instrumentation: [76e40add] regParam is zero, which might cause numerical instability and overfitting.


In [16]:
print(model2.coefficients)
print('rmse:', model2.summary.rootMeanSquaredError)
print('mae:', model2.summary.meanAbsoluteError)
print('r2', model2.summary.r2)

[-42173.45338895586,-42081.551257577135,1133.8249791510773,-7.437950377894255,80.60555228964593,-38.459232850866634,81.15642504503433,39898.34472732782]
rmse: 69676.55888505312
mae: 50862.62426319554
r2 0.633719411250925


## Анализ результатов <a id="4"></a>

Применение моделей на тестовой выборке:

In [17]:
predictions = model.transform(datatest)
predictions.select("prediction", "label", "features").show(5)

evaluator = RegressionEvaluator()
rmse = evaluator.evaluate(predictions)
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
print(rmse)
print(r2)
print(mae)

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|190400.39422662533|111400.0|[-62.152092855925...|
|198108.56428025407| 90100.0|[-62.117080410247...|
|176489.41627548845| 69000.0|[-62.117080410247...|
|147980.29676334956| 70000.0|[-62.117080410247...|
|189445.67376456806|107000.0|[-62.112078632293...|
+------------------+--------+--------------------+
only showing top 5 rows

68744.17100878287
0.6512649624726952
49956.13783437518


In [18]:
predictions2 = model2.transform(datatest)
predictions2.select("prediction", "label", "features").show(5)

rmse = evaluator.evaluate(predictions2)
r2 = evaluator.evaluate(predictions2, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(predictions2, {evaluator.metricName: "mae"})
print(rmse)
print(r2)
print(mae)

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|163334.31861887267|111400.0|[-62.152092855925...|
|165487.47869401937| 90100.0|[-62.117080410247...|
| 144213.4960800414| 69000.0|[-62.117080410247...|
|115366.31847691815| 70000.0|[-62.117080410247...|
| 159791.9323567045|107000.0|[-62.112078632293...|
+------------------+--------+--------------------+
only showing top 5 rows

69617.04280997363
0.6423526859730702
51032.132464003414


Качество модели, обученной на всех данных немного выше. В целом, коэффициент детерминации показывает, что обе модели хорошо объясняют данные.

In [19]:
spark.stop()

## Общий вывод <a id="5"></a>

В ходе исследования данные были раздены на две выборки: выборку, состоящую только из числовых данных, и выборку, включающую категориальные данные.

Данные были подготовлены для обучения модели: заполнены пропуски, проведено кодирование категориальных и масштабирование числовых, а также векторизация данных.

Модель линейной регрессии обучена на двух наборах данных. Значение метрики r2 выше на 0.01 при обучении на всех данных. Значения метрик r2, rmse, mae в этом случае равны 0.65, 68744 и 49956 соответсвенно.
